In [1]:
import pandas as pd
import re

pd.set_option('display.max_columns', None)

In [2]:
db = pd.read_csv(r'C:\Users\eduar\OneDrive\Desktop\DEV\Football\SerieA\database.csv',index_col=None)

In [5]:
db

,Player,Team,#,Nation,Position,Age,Minutes,Goals,Assists,Penalty Shoot on Goal,Penalty Shoot,Total Shoot,Shoot on Target,Yellow Cards,Red Cards,Touches,Dribbles,Tackles,Blocks,Expected Goals (xG),Non-Penalty xG (npxG),Expected Assists (xAG),Shot-Creating Actions,Goal-Creating Actions,Passes Completed,Passes Attempted,Pass Completion %,Progressive Passes,Carries,Progressive Carries,Dribble Attempts,Successful Dribbles,Date
0,Vitinha,Genoa,9,POR,FW,24-155,90,0,0,0,0,3,1,0,0,28,0,0,2,0.2,0.2,0.0,5,0,8,13,"61,5",1,16,3,6,5,2024-08-17
1,Junior Messias,Genoa,10,BRA,"FW,RW",33-096,90,1,0,0,1,2,2,0,0,23,0,1,0,1.0,0.0,0.1,4,1,6,8,"75,0",1,13,1,4,1,2024-08-17
2,Morten Frendrup,Genoa,32,DEN,CM,23-132,90,0,0,0,0,1,1,0,0,34,1,3,2,0.0,0.0,0.0,1,0,20,23,"87,0",3,13,0,1,1,2024-08-17
3,Milan Badelj,Genoa,47,CRO,CM,35-174,85,0,0,0,0,1,0,0,0,38,1,0,2,0.1,0.1,0.1,2,0,23,28,"82,1",2,20,0,2,1,2024-08-17
4,Jeff Ekhator,Genoa,21,ITA,LW,17-280,5,0,0,0,0,0,0,0,0,5,0,0,0,0.0,0.0,0.0,0,0,1,2,"50,0",0,2,0,0,0,2024-08-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2498,Danilo D'Ambrosio,Monza,33,ITA,WB,36-042,19,0,0,0,0,0,0,0,0,5,0,0,1,0.0,0.0,0.0,0,0,4,4,"100,0",0,1,0,0,0,2024-10-21
2499,Andrea Carboni,Monza,44,ITA,CB,23-260,90,0,0,0,0,0,0,1,0,54,2,0,4,0.0,0.0,0.0,2,1,29,40,"72,5",3,29,0,2,1,2024-10-21
2500,Pablo Marí,Monza,22,ESP,CB,31-051,90,0,0,0,0,0,0,0,0,60,2,2,1,0.0,0.0,0.0,1,0,42,49,"85,7",1,24,0,0,0,2024-10-21
2501,Armando Izzo,Monza,4,ITA,CB,32-233,90,0,0,0,0,0,0,0,0,48,2,1,3,0.0,0.0,0.0,0,0,29,35,"82,9",1,18,0,0,0,2024-10-21


In [6]:
# Agrupar os dados por jogador e calcular a soma dos chutes ao gol e a contagem de aparições
db_agrupado = db.groupby(['Player','Team']).agg(
    Total_Gols=('Goals','sum'),
    Total_Chutes=('Total Shoot', 'sum'),   # Soma dos chutes ao gol
    Aparicoes=('Player', 'count'), # Contagem de aparições
    Total_Chutes_Gol=('Shoot on Target', 'sum'), # Contagem de aparições
).sort_values(by='Total_Chutes', ascending=False)  # Ordenar antes do reset_index

# Adicionar uma coluna de ranking
db_agrupado['Ranking'] = db_agrupado['Total_Chutes'].rank(method='min', ascending=False).astype(int)

# Adicionar uma coluna de média (Total_Chutes / Aparicoes)
db_agrupado['Média_Chutes'] = round(db_agrupado['Total_Chutes'] / db_agrupado['Aparicoes'],2)

db_agrupado['Média_Chutes_Gol'] = round(db_agrupado['Total_Chutes_Gol'] / db_agrupado['Aparicoes'],2)

# Resetar o índice
db_agrupado.reset_index(inplace=True)

# Reorganizar as colunas para que o Ranking seja a primeira
db_agrupado = db_agrupado[['Ranking', 'Player','Team' ,'Aparicoes','Total_Gols','Total_Chutes','Média_Chutes','Total_Chutes_Gol','Média_Chutes_Gol']]

In [10]:
db_agrupado[db_agrupado['Team'].isin(['Internazionale'])]

,Ranking,Player,Team,Aparicoes,Total_Gols,Total_Chutes,Média_Chutes,Total_Chutes_Gol,Média_Chutes_Gol
7,8,Marcus Thuram,Internazionale,8,7,23,2.88,14,1.75
15,14,Lautaro Martínez,Internazionale,7,3,20,2.86,8,1.14
16,17,Federico Dimarco,Internazionale,8,1,19,2.38,3,0.38
51,47,Davide Frattesi,Internazionale,8,1,10,1.25,1,0.12
93,80,Hakan Çalhanoğlu,Internazionale,7,1,7,1.00,2,0.29
101,102,Matteo Darmian,Internazionale,8,1,6,0.75,1,0.12
108,102,Henrikh Mkhitaryan,Internazionale,8,0,6,0.75,0,0.00
118,102,Nicolò Barella,Internazionale,5,1,6,1.20,1,0.20
180,160,Mehdi Taremi,Internazionale,6,0,4,0.67,2,0.33
188,160,Denzel Dumfries,Internazionale,6,1,4,0.67,4,0.67


In [11]:
filtro = db_agrupado['Team'].isin(['Internazionale','Venezia'])
# Selecionar os 10 jogadores com o maior número de chutes
top_10_chutadores = db_agrupado[filtro].nlargest(10, 'Média_Chutes')
top_10_chutadores

,Ranking,Player,Team,Aparicoes,Total_Gols,Total_Chutes,Média_Chutes,Total_Chutes_Gol,Média_Chutes_Gol
7,8,Marcus Thuram,Internazionale,8,7,23,2.88,14,1.75
15,14,Lautaro Martínez,Internazionale,7,3,20,2.86,8,1.14
16,17,Federico Dimarco,Internazionale,8,1,19,2.38,3,0.38
50,47,Gianluca Busio,Venezia,5,1,10,2.00,3,0.60
25,25,Gaetano Oristanio,Venezia,8,1,15,1.88,5,0.62
63,55,Ridgeciano Haps,Venezia,5,0,9,1.80,1,0.20
51,47,Davide Frattesi,Internazionale,8,1,10,1.25,1,0.12
118,102,Nicolò Barella,Internazionale,5,1,6,1.20,1,0.20
95,80,Hans Nicolussi Caviglia,Venezia,6,0,7,1.17,3,0.50
98,80,Joel Pohjanpalo,Venezia,6,2,7,1.17,3,0.50


In [14]:
db[db['Player']=='Lautaro Martínez']

,Player,Team,#,Nation,Position,Age,Minutes,Goals,Assists,Penalty Shoot on Goal,Penalty Shoot,Total Shoot,Shoot on Target,Yellow Cards,Red Cards,Touches,Dribbles,Tackles,Blocks,Expected Goals (xG),Non-Penalty xG (npxG),Expected Assists (xAG),Shot-Creating Actions,Goal-Creating Actions,Passes Completed,Passes Attempted,Pass Completion %,Progressive Passes,Carries,Progressive Carries,Dribble Attempts,Successful Dribbles,Date
16,Lautaro Martínez,Internazionale,10,ARG,"FW,AM",26-361,85,0,0,0,0,1,1,0,0,25,0,0,1,0.3,0.3,0.0,3,0,14,18,"77,8",2,17,0,1,0,2024-08-17
655,Lautaro Martínez,Internazionale,10,ARG,FW,27-008,82,0,0,0,0,1,0,0,0,26,1,0,0,0.1,0.1,0.0,2,0,16,21,"76,2",2,24,1,2,1,2024-08-30
1170,Lautaro Martínez,Internazionale,10,ARG,FW,27-024,55,0,0,0,0,2,0,0,0,16,0,0,0,0.2,0.2,0.0,0,0,6,9,"66,7",0,9,0,1,0,2024-09-15
1499,Lautaro Martínez,Internazionale,10,ARG,FW,27-031,90,0,1,0,0,3,1,0,0,31,0,1,0,0.2,0.2,0.2,5,2,14,22,"63,6",2,17,1,2,1,2024-09-22
1609,Lautaro Martínez,Internazionale,10,ARG,FW,27-037,85,2,0,0,0,4,3,0,0,27,1,0,0,0.7,0.7,0.7,3,0,13,14,"92,9",3,15,2,4,1,2024-09-28
2003,Lautaro Martínez,Internazionale,10,ARG,FW,27-044,90,0,0,0,0,6,1,0,0,33,4,0,0,0.4,0.4,0.0,1,1,18,22,"81,8",2,19,1,1,1,2024-10-05
2455,Lautaro Martínez,Internazionale,10,ARG,FW,27-059,71,1,0,0,0,3,2,0,0,26,0,0,0,0.2,0.2,0.1,3,0,11,18,"61,1",3,17,2,2,0,2024-10-20
